In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import cv2
import os.path


ModuleNotFoundError: No module named 'torch'

` FACD_image - contains directories of the original and filtered images
* original images - Origin
* filtered images - 1977, Amaro, Apollp, Brannan, Earlybird, Gotham, Hefe, Hudson, Inkwell, Lofi, LordKevin, Mayfair, Nashville, Poprocket, Rise, Sierra, Sutro, Toaster, Valencia, Walden, Willow, and XProII
FACD_metadata - contains two pkl files about the information of the images and the pairs
* pairwise_comparison.pkl
    * category - image category
    * imgId - image ID
    * f1 - the first filter
    * f2 - the second filter
    * ans - user preference between two filtered images
    * score - the aesthetic score computed from comparison results
    * class - high or low quality
* image_score.pkl
    * filterName - filter name
    * imgId - image ID
    * score - the aesthetic score computed from comparison results
    * class - high or low quality`

In [2]:
pickle_1 = open("FACD_metadata/pairwise_comparison.pkl","rb")
pickle_2 = open("FACD_metadata/image_score.pkl","rb")
comparison_pickle = pickle.load(pickle_1)
image_score = pickle.load(pickle_2)

In [3]:
comparison_pickle = pd.DataFrame(comparison_pickle)
image_score = pd.DataFrame(image_score)
print(len(comparison_pickle))
print(len(image_score))
comparison_pickle.head(n=5)

42240
28160


,category,f1,f2,workerId,passDup,imgId,ans
0,2,Willow,1977,A34M93NJC830DP,True,755006,right
1,1,Toaster,Valencia,A34M93NJC830DP,True,451445,right
2,3,Apollo,Brannan,A34M93NJC830DP,True,749452,right
3,4,Sutro,Walden,A1PCTP576OW35H,True,653868,left
4,0,1977,Brannan,A1PCTP576OW35H,True,55771,left


In [4]:
print(set(image_score['score']))
image_score.head()

{0, 1, 2, 3, -2, -3, -1}


,filterName,imgId,class,score
0,Brannan,137059,1,3
1,Willow,137059,0,-1
2,Mayfair,137059,1,3
3,Hudson,137059,0,-1
4,Gotham,137059,1,3


In [5]:
category_0 = comparison_pickle[comparison_pickle['category']==0]
category_0 = category_0[category_0['ans'].isin(["left", 'right'])]
print(len(category_0))
category_0.head()

4964


,category,f1,f2,workerId,passDup,imgId,ans
4,0,1977,Brannan,A1PCTP576OW35H,True,55771,left
18,0,XProII,Nashville,AAOTQL1SCAC03,True,606762,left
22,0,Hudson,Apollo,AAOTQL1SCAC03,True,338983,left
26,0,Lofi,Hefe,ANPCXN619ACW9,True,713650,right
51,0,Valencia,Amaro,A15RDD1EQIZSX3,True,124458,left


In [10]:
def image_to_tensor(file_path_list, size = (3, 224, 224)):
    output = []
    for file_path in file_path_list:
        im= np.asarray(Image.open(file_path).convert('RGB'))
        im=cv2.resize(im, dsize=(size[1],size[2]))
#         print(im.shape)
        im = np.reshape(im, size)
        output.append(im)
    return output

def find_image(filter_name, image_id):
    for file in os.listdir("./FACD_image/"+ filter_name):
        if image_id in file:
            return "./FACD_image/"+ filter_name + "/" + str(image_id) + ".jpg"
        
def generate_comparison_data(category_df):
    comp1 = []
    comp2 = []
    comp_labels =[]
    labels_map = {"left": 1, "right": 0}
    for index, row in category_df.iterrows():
        img_id = row['imgId']
        comp1_filter = row['f1']
        comp2_filter = row['f2']
        fname1 = "./FACD_image/"+ comp1_filter + "/" + str(img_id) + ".jpg"
        fname2 = "./FACD_image/"+ comp2_filter + "/" + str(img_id) + ".jpg"
        if os.path.isfile(fname1) and os.path.isfile(fname2):
            comp1.append(fname1)
            comp2.append(fname2)
            comp_labels.append(labels_map[row['ans']])
    return comp1, comp2, comp_labels
            
# generate_comparison_data(category_0)

In [9]:
category_0 = comparison_pickle[comparison_pickle['category']==0]
winning_filters = []
for index, row in category_0.iterrows():
    if row['ans'] == "left":
        winning_filters.append(row['f1'])
    else:
        winning_filters.append(row['f2'])

In [10]:
image_id_score_map = {}
imgid = list(image_score["imgId"])
filters = list(image_score["filterName"])
scores = list(image_score["score"])
for i in range(0, len(imgid)):
    img_i = imgid[i]
    score_i = scores[i]
    filter_i = filters[i]
    image_id_score_map[(img_i,filter_i)] = score_i
    


score_column = []
count = 0
for index, row in category_0.iterrows():
    img = row['imgId']
    winning_filter = winning_filters[count]
    to_add = image_id_score_map[img, winning_filter]
    count += 1
    score_column.append(to_add)

In [11]:
category_0["score"]=score_column
category_0.head()

<ipython-input-11-ef47751bb536>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  category_0["score"]=score_column


,category,f1,f2,workerId,passDup,imgId,ans,score
4,0,1977,Brannan,A1PCTP576OW35H,True,55771,left,3
11,0,Gotham,Earlybird,A1PCTP576OW35H,True,386053,equal,2
18,0,XProII,Nashville,AAOTQL1SCAC03,True,606762,left,-1
22,0,Hudson,Apollo,AAOTQL1SCAC03,True,338983,left,2
26,0,Lofi,Hefe,ANPCXN619ACW9,True,713650,right,3


In [14]:
len(category_0)
abs_images = []
abs_labels = []
for index, row in category_0.iterrows():
    img_id = row["imgId"]
    if row['ans'] == "left":
        f1 = row['f1']
        fname1 = "./FACD_image/"+ f1 + "/" + str(img_id) + ".jpg"
        if os.path.isfile(fname1):
            abs_images.append(fname1)
            abs_labels.append(row['score'])
    else:
        f2 = row['f2']
        fname2 = "./FACD_image/"+ f2 + "/" + str(img_id) + ".jpg"
        if os.path.isfile(fname2):
            abs_images.append(fname2)
            abs_labels.append(row['score'])

In [20]:
print(len(comparison_pickle['imgId']))
print(len(set(comparison_pickle['imgId'])))
print(len(image_score['imgId']))

42240
1280
28160


In [25]:

category_0.head()

,category,f1,f2,workerId,passDup,imgId,ans
4,0,1977,Brannan,A1PCTP576OW35H,True,55771,left
11,0,Gotham,Earlybird,A1PCTP576OW35H,True,386053,equal
18,0,XProII,Nashville,AAOTQL1SCAC03,True,606762,left
22,0,Hudson,Apollo,AAOTQL1SCAC03,True,338983,left
26,0,Lofi,Hefe,ANPCXN619ACW9,True,713650,right


In [82]:
print(set(category_0['score']))
category_0_ = category_0[category_0['score'].isin([-3, 2, 3, 2, 1, -1])]
print(len(category_0_))
category_0_.head()

{0, 1, 2, 3, -2, -1}
4864


,category,f1,f2,workerId,passDup,imgId,ans,score
1,1,Toaster,Valencia,A34M93NJC830DP,True,451445,right,-1
5,1,Amaro,Willow,A1PCTP576OW35H,True,695877,left,1
23,1,Apollo,Mayfair,ANPCXN619ACW9,True,791689,right,1
24,1,Brannan,Earlybird,ANPCXN619ACW9,True,824571,right,3
28,1,Sutro,Apollo,ANPCXN619ACW9,True,664173,left,1


In [57]:
comparison_pickle_test= image_score[image_score['score'].isin([-3,3])]
print(len(comparison_pickle_test))

9327


In [13]:
print(len(set(image_score['imgId'])))
print(len(set(comparison_pickle['category'])))
print(len(comparison_pickle[comparison_pickle['category']==0]))
print(len(image_score[image_score['filterName']=="1977"]))

# comparison_pickle = comparison_pickle[comparison_pickle['score']==3]
len_a= len(comparison_pickle[comparison_pickle['f1']=="1977"])
len_b =len(comparison_pickle[comparison_pickle['f2']=="1977"])
print(len_a, len_b, len_a + len_b)

604
8
5280
0
1932 1908 3840


In [8]:
image_score = image_score[image_score['filterName']=="Valencia"]
image_score.head()

,filterName,imgId,class,score
7,Valencia,137059,0,-3
29,Valencia,696485,0,0
51,Valencia,182772,0,-3
73,Valencia,911972,0,-1
95,Valencia,380038,0,-3


In [9]:
image_score= image_score[image_score['score'].isin([-3,3])]
image_score.head()

,filterName,imgId,class,score
7,Valencia,137059,0,-3
51,Valencia,182772,0,-3
95,Valencia,380038,0,-3
139,Valencia,200444,0,-3
205,Valencia,818632,0,-3


In [10]:
len(image_score)

604

In [ ]:
comparison_picklef1= comparison_pickle[comparison_pickle['f1']=="Valencia"]
comparison_picklef2=comparison_pickle= comparison_pickle[comparison_pickle['f1']=="Valencia"]

In [30]:
print(len(image_score['imgId']))
print(len(comparison_pickle['imgId']))

28160
42240


In [31]:
image_score.head(n=5)

,filterName,imgId,class,score
0,Brannan,137059,1,3
1,Willow,137059,0,-1
2,Mayfair,137059,1,3
3,Hudson,137059,0,-1
4,Gotham,137059,1,3


In [8]:
true_categories = set(os.listdir("./FACD_image"))
categories_f1 = set(comparison_pickle['f1'])
categories_f2 = set(comparison_pickle['f2'])
print(true_categories)

{'Sutro', 'Hudson', 'Amaro', 'Rise', '1977', 'Apollo', 'XProII', 'Nashville', 'Walden', 'Lofi', 'Earlybird', 'Willow', 'Origin', 'Hefe', 'Gotham', 'Sierra', 'Inkwell', 'Brannan', 'Valencia', 'Toaster', 'Mayfair', 'LordKevin', '.DS_Store', 'Poprocket'}


In [9]:
print(categories_f1 == categories_f2)
print(true_categories - categories_f1)

True
{'.DS_Store', 'Origin'}


In [10]:
def find_image(category, image_id):
    for file in os.listdir("./FACD_image/"+ category):
        if image_id in file:
#             print("./FACD_image/"+ category + "/" + str(image_id) + ".jpg")
            return "./FACD_image/"+ category + "/" + str(image_id) + ".jpg"

Filter Aesthetic Comparison Dataset (FACD) contains 1,280 unfiltered images collected from the AVA dataset within 8 categories. It also includes 28,160 filtered images and 42,240 image pairs. 

In [11]:
print(len(image_score))
print(len(comparison_pickle))
print(len(os.listdir('./FACD_image/Origin')))

28160
42240
1280


In [12]:
f1 = comparison_pickle['f1']
f2 = comparison_pickle['f2']
image_id = comparison_pickle['imgId']
f1_zip = list(zip(f1, image_id))
f2_zip = list(zip(f2, image_id))

### abs_imgs, abs_labels, comp_imgs_1, comp_imgs_2, comp_labels = load_data(datasplit='train')

In [13]:
comp_imgs_1 = [find_image(c, i) for c, i in f1_zip]
len(comp_imgs_1)

<function len(obj, /)>

In [14]:
comp_imgs_2 = [find_image(c, i) for c, i in f2_zip]
len(comp_imgs_2)

42240

(i,j, y): given items (i, j), y(i,j) = +1 indicates that i has a higher propensity to receive the absolute label yi = +1,
compared to j. -1 otherwise.

In [15]:
comp_labels = [1 if i=='left' else -1 for i in comparison_pickle['ans']]
print(len(comp_labels))

42240


In [16]:
abs_imgs = ['./FACD_image/Origin/' + file for file in os.listdir('./FACD_image/Origin')]
print(len(abs_imgs))

1280


In [14]:
abs_image_class_map = {}
for index, row in image_score.iterrows():
    if row['imgId'] not in abs_image_class_map:
        abs_image_class_map[row['imgId']] = row['class']

In [15]:
abs_labels = [abs_image_class_map[file[:-4]] for file in os.listdir('./FACD_image/Origin')]

In [21]:
def image_to_tensor(file_path_list, size = (3, 224, 224)):
    output = []
    for file_path in file_path_list:
        im= np.asarray(Image.open(file_path).convert('RGB'))
        im=cv2.resize(im, dsize=(size[1],size[2]))
#         print(im.shape)
        im = np.reshape(im, size)
        output.append(im)
    return output

In [22]:
a = image_to_tensor(abs_imgs)


/Users/dipika/.pyenv/versions/3.7.9/envs/home/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19273 bytes but only got 816. Skipping tag 700
  "Possibly corrupt EXIF data.  "
/Users/dipika/.pyenv/versions/3.7.9/envs/home/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5140 bytes but only got 816. Skipping tag 34377
  "Possibly corrupt EXIF data.  "
/Users/dipika/.pyenv/versions/3.7.9/envs/home/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:771: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3144 bytes but only got 816. Skipping tag 34675
  "Possibly corrupt EXIF data.  "


In [23]:
np.shape(a)

(1280, 3, 224, 224)

# Should There Be 8 Classes


In [26]:
pickle_1 = open("FACD_metadata/pairwise_comparison.pkl","rb")
pickle_2 = open("FACD_metadata/image_score.pkl","rb")
comparison_pickle = pickle.load(pickle_1)
image_score = pickle.load(pickle_2)

In [27]:
comparison_pickle = pd.DataFrame(comparison_pickle)
image_score = pd.DataFrame(image_score)
comparison_pickle.head(n=5)

,category,f1,f2,workerId,passDup,imgId,ans
0,2,Willow,1977,A34M93NJC830DP,True,755006,right
1,1,Toaster,Valencia,A34M93NJC830DP,True,451445,right
2,3,Apollo,Brannan,A34M93NJC830DP,True,749452,right
3,4,Sutro,Walden,A1PCTP576OW35H,True,653868,left
4,0,1977,Brannan,A1PCTP576OW35H,True,55771,left


In [28]:
image_score.head(n=5)

,filterName,imgId,class,score
0,Brannan,137059,1,3
1,Willow,137059,0,-1
2,Mayfair,137059,1,3
3,Hudson,137059,0,-1
4,Gotham,137059,1,3


In [29]:
true_categories = set(os.listdir("./FACD_image"))
categories_f1 = set(comparison_pickle['f1'])
categories_f2 = set(comparison_pickle['f2'])
print(true_categories)

{'Sutro', 'Earlybird', 'Apollo', 'Mayfair', 'Hefe', 'Origin', 'Poprocket', 'Lofi', 'Brannan', 'LordKevin', 'Sierra', 'Valencia', 'Toaster', 'Nashville', 'Hudson', '1977', 'XProII', 'Rise', 'Walden', 'Gotham', 'Willow', '.DS_Store', 'Amaro', 'Inkwell'}


In [30]:
print(categories_f1 == categories_f2)
print(true_categories - categories_f1)

True
{'Origin', '.DS_Store'}


In [31]:
def find_image(category, image_id):
    for file in os.listdir("./FACD_image/"+ category):
        if image_id in file:
#             print("./FACD_image/"+ category + "/" + str(image_id) + ".jpg")
            return "./FACD_image/"+ category + "/" + str(image_id) + ".jpg"

In [32]:
print(len(image_score))
print(len(comparison_pickle))
print(len(os.listdir('./FACD_image/Origin')))

28160
42240
1280


In [33]:
f1 = comparison_pickle['f1']
f2 = comparison_pickle['f2']
image_id = comparison_pickle['imgId']
f1_zip = list(zip(f1, image_id))
f2_zip = list(zip(f2, image_id))

In [37]:
comp_imgs_1 = [find_image(c, i) for c, i in f1_zip][:4]
comp_imgs_2 = [find_image(c, i) for c, i in f2_zip][:4]
comp_labels = [1 if i=='left' else -1 for i in comparison_pickle['ans']][:4]

KeyboardInterrupt: 

In [38]:
abs_imgs = ['./FACD_image/Origin/' + file for file in os.listdir('./FACD_image/Origin')]

In [39]:
abs_image_class_map = {}
for index, row in image_score.iterrows():
    if row['imgId'] not in abs_image_class_map:
        abs_image_class_map[row['imgId']] = row['filterName']

In [50]:
abs_labels = [abs_image_class_map[file[:-4]] for file in os.listdir('./FACD_image/Origin')]
set(abs_labels)

{'Brannan'}

# OTHER

In [5]:
# def load_images(image_size=32, batch_size=64, root="../images"):
#     transform = transforms.Compose([
#         transforms.Resize(32),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#     train_set = datasets.ImageFolder(root=root, train=True, transform=transform)
#     train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

#     return train_loader

def load_data(path):
    train_dataset = torchvision.datasets.ImageFolder(
        root=path,
        transform=transforms.Compose([
        transforms.Resize([512,512]),
        transforms.ToTensor()])
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        num_workers=0,
        shuffle=True
    )
    return train_loader

def create_googlenet(input):
    model = models.googlenet(pretrained=True)
    return model

In [6]:
path = "FACD_image"
data_loader = load_data(path)

In [7]:
print(data_loader)
for i in data_loader:
    print(i[1])
    print(len(i))
    print(np.shape(i[0]))
    break

tensor([20])
2
torch.Size([1, 3, 512, 512])


In [8]:
import torchvision.models as models

In [9]:
googlenet = models.googlenet(pretrained=True)

In [10]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open("FACD_image/1977/2729.jpg")
model = models.googlenet(pretrained=True)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
print(torch.nn.functional.softmax(output[0], dim=0))

tensor([-3.1899e-02,  3.5153e-01,  1.8458e-01, -1.4896e-01, -4.6124e-01,
        -8.4455e-01, -5.1574e-01, -1.9230e-01, -2.2986e-02, -5.6370e-01,
        -5.2641e-01, -9.4004e-01, -4.1236e-01, -4.1846e-01, -9.0512e-01,
        -1.3300e+00, -1.1582e+00,  7.6318e-02, -2.4474e-01, -5.8722e-01,
        -2.4835e-01, -4.3553e-01, -1.5566e-01, -3.8703e-01,  3.9863e-01,
        -1.8854e-01, -5.7036e-01, -6.9493e-01,  2.0408e-01, -1.9567e-01,
         4.0233e-03, -1.2297e-01,  6.0439e-02, -1.4385e-01, -1.5953e-01,
        -2.5152e-01, -2.2263e-01, -3.2749e-01, -2.0231e-03, -4.4885e-01,
        -6.3140e-01, -3.8229e-01, -3.2429e-01, -2.5189e-01, -1.7986e-01,
         1.9626e-01, -2.5517e-01, -7.5576e-01, -2.8026e-01, -4.6869e-02,
        -1.8780e-01,  3.6231e-01, -3.7293e-01,  1.4561e-01, -5.8026e-01,
        -1.7333e-01, -6.4085e-02, -3.8692e-01,  9.3555e-02, -4.0926e-01,
         2.3060e-01,  7.3190e-02,  2.1534e-01,  2.5186e-01,  2.5822e-01,
         1.2012e-01, -1.8018e-01, -9.2184e-02, -3.3

In [4]:
def create_siamese(reg_param=0.0002, no_of_score_layers=1, max_no_of_nodes=128):
    input1 = Input(shape=self.input_shape)
    input2 = Input(shape=self.input_shape)
    print("input 1: ", input1)
    print("input 2: ", input2)
    # get features from base network
    feature1 = Dense(self.no_of_classes, activation='softmax', kernel_regularizer=l2(reg_param), name='abs1')
    feature2 = Dense(self.no_of_classes, activation='softmax', kernel_regularizer=l2(reg_param), name='comp1')
    # create and pass through score layers
    score1 = feature1
    score2 = feature2
    for l in range(no_of_score_layers-1):
        layer = Dense(int(max_no_of_nodes / (l+2)), activation='relu', kernel_regularizer=l2(reg_param), name='s'+str(l))
        score1 = layer(score1)
        score2 = layer(score2)
    # create final layers of absolute and comparison
    abs_out = Dense(self.no_of_classes, activation='softmax', kernel_regularizer=l2(reg_param), name='abs')
    comp_out = Dense(1, activation='sigmoid', kernel_regularizer=l2(reg_param), name='comp')
    # absolute part
    abs_out1 = abs_out(score1)
    abs_net = Model(input1, abs_out1)
    # comparison part
    comp_out1 = comp_out(score1)
    comp_out2 = comp_out(score2)
    distance = Lambda(BTPred, output_shape=(1,))([comp_out1, comp_out2])
    comp_net = Model([input1, input2], distance)
    return abs_net, comp_net

In [5]:
create_siamese()

NameError: name 'Input' is not defined

In [19]:




def creat_siamese2(no_of_score_layers=2, max_no_of_nodes=128, absolute_data=None):
    model_ft = create_googlenet()

    return model_ft
        
    # create final layers of absolute and comparison

ImportError: cannot import name 'Layer' from 'keras.layers.core' (/Users/dipika/.pyenv/versions/3.7.9/envs/home/lib/python3.7/site-packages/keras/layers/core.py)